In [240]:
import pandas as pd
from curl_cffi import requests
import sqlite3

In [143]:
h = {
 'Accept': 'application/json, text/plain, */*',
'Accept-Encoding': 'gzip, deflate, br, zstd',
#'Cookie':'tmr_lvid=35d215fb3a77569be984da1d45f85d55; tmr_lvidTS=1720105202049; _ym_uid=1720105202382995222; _ym_d=1720105202; spid=1720105288467_5404a7e4c3f2f43831232762826fc282_d8j0eti32peevdul; _ym_isad=2; spsc=1720308606850_c99ebfa602e2614aff5876336b298206_2dc4c47e5beb4aae25be080fa9d16c8093e7e989cef732b63b8bada59af3d7da',
'Referer':'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82/60683',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

In [91]:
def get_date(offset):
    ###Выгрузка списка объектов
    url =f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset}&limit={100}&sortField=obj_publ_dt&sortType=desc&residentialBuildings=1&objStatus=0'
    res = requests.get(url = url, headers= h, impersonate= 'chrome116')
    return res.json()['data']['list'], res.json()['data']['total']

In [188]:
def data_obj(obj_id):
    ###Выгрузка информации по объекту + инфраструктура.
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj_id}'
    url_in = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj_id}/infrastructure'
    l = []
    res = requests.get(url = url, headers= h, impersonate= 'chrome116')
    try:
        res_in = requests.get(url = url_in, headers= h, impersonate= 'chrome116')
    except Exception:
        print("error", obj_id)
    try:
        df = pd.json_normalize(res.json()['data'])
    except Exception:
        l.append(obj_id)
        df = pd.DataFrame()
        print(res, 'error', 'obj')
    try:
        df_in = pd.json_normalize(res_in.json()['data'])
        df_in['id'] = obj_id
        df_in = df_in[['tabName','id']]
    except Exception:
        print('No data', obj_id)
        df_in = pd.DataFrame()
    return df, df_in, l

In [97]:
offset = 0
list_obj = []
while True:
    data, total = get_date(offset=offset)
    if len(data) == 0:
        break
    else:
        obj = [i.get('objId') for i in data]
        list_obj.extend(obj)
        if offset % 1000 ==0:
            print('Выгружено', offset, ' строк')
        offset += 100
        
print('Выгружено ',len(list_obj), ' из',total)        

Выгружено 0  строк
Выгружено 1000  строк
Выгружено 2000  строк
Выгружено 3000  строк
Выгружено 4000  строк
Выгружено 5000  строк
Выгружено 6000  строк
Выгружено 7000  строк
Выгружено 8000  строк
Выгружено 9000  строк
Выгружено 10000  строк
Выгружено 11000  строк
Выгружено  11211  из 11211


In [189]:
df_data = pd.DataFrame()
df_inf = pd.DataFrame(),203 + 1613
for obj in tqdm(list_obj[1818:]):
    data, inf, l = data_obj(obj)
    df_data = pd.concat([df_data,data])
    df_inf = pd.concat([df_inf,inf])
df_data.shape

  0%|          | 0/9393 [00:00<?, ?it/s]

No data 58087
No data 57086
No data 57071
No data 57014
No data 54868
No data 54163
No data 53350
No data 50907
No data 50906
No data 50559
No data 50092
No data 48984


(11211, 115)

In [230]:
df_inf_new = df_inf[df_inf.tabName.isin(['Образование','Медицина','Продукты'])]

In [ ]:
df_inf_new['coun'] = 1

In [238]:
pd.DataFrame.to_excel(df_inf_new.groupby(['id','tabName']).count(),'Hometask_infr.xlsx', index = False)

In [239]:
pd.DataFrame.to_pickle(df_data,'Hometask.pkl')

In [270]:
conn = sqlite3.connect('test_database')

In [273]:
col = ['id',
 'hobjId',
 'pdId',
 'region',
 'address',
 'objPublDt',
 'floorMin',
 'floorMax',
 'objElemLivingCnt',
 'objReady100PercDt',
 'wallMaterialShortDesc',
 'objLkFinishTypeCount',
 'objLkFreePlanDesc',
 'objElemParkingCnt',
 'objSquareLiving',
 'rpdNum',
 'rpdPdfLink',
 'rpdIssueDttm',
 'objLkLatitude',
 'objLkLongitude',
 'objCompensFundFlg',
 'objProblemFlg',
 'objLkClassDesc']

In [ ]:
###Запись всего Dataframe выдавло ошибку

df[col].to_sql('products', conn, if_exists='replace', index = False)
conn.commit()
conn.close()